In [7]:
import os
import pandas as pd

DATAPATH = "../data"
CONFIGPATH = "../config"

In [9]:
allunits = []

# List all files in the folder
file_list = os.listdir(os.path.join(DATAPATH, "pathogen_original"))

# Iterate through each file
for filename in file_list:
    df = pd.read_csv(os.path.join(DATAPATH, "pathogen_original", filename), low_memory=False)
    units = list(set(df["standard_units"]))
    allunits.extend(units)

allunits = list(set(allunits))
df = pd.DataFrame({"units": allunits})
df.to_csv(os.path.join(CONFIGPATH, "units.csv"), index=False)

In [22]:
df = pd.read_csv(os.path.join(DATAPATH, "pathogen_original", "efaecium.csv"), low_memory=False)
df.columns

Index(['doc_id', 'assay_id', 'activity_id', 'assay_type',
       'assay_confidence_score', 'assay_bao_format', 'compound_chembl_id',
       'canonical_smiles', 'standard_type', 'standard_value', 'standard_units',
       'standard_relation', 'pchembl_value', 'activity_comment',
       'target_chembl_id', 'target_type', 'target_organism',
       'target_pref_name', 'target_tax_id', 'protein_accession_class', 'year',
       'pubmed_id', 'assay_chembl_id', 'count_activity_rows', 'doc_id_all',
       'assay_id_all', 'activity_id_all', 'assay_description'],
      dtype='object')

In [13]:
#the units file has been processed in UCUM
#selected units will be converted to standard:
#Molar: umol
#weight/volume: ug.ml-1 to uM
#weight/weight: ug.mg-1
#molar/weight: umol/mg
f = "lambda x, y: x*y/1000"

f = eval(f)

In [15]:
f(3,2)

0.006

In [18]:
s = 'standard_value/(molecular_weight/1000)/1000'

def parse_function(s):
    if 'standard_value' not in s:
        return None
    if "molecular_weight" in s:
        p = "lambda x,y: "
    else:
        p = "lambda x: "
    s = s.replace("molecular_weight", "y")
    s = s.replace("standard_value", "x")
    s = p + s
    return eval(s)

f = parse_function(s)

In [19]:
f(1,2)

0.5